In [1]:
!curl -LO "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M    0 58.1M    0     0  40.4M      0 --:--:--  0:00:01 --:--:-- 40.4M


In [2]:
!unzip human+activity+recognition+using+smartphones.zip

Archive:  human+activity+recognition+using+smartphones.zip
replace UCI HAR Dataset.names? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!unzip -qq "UCI HAR Dataset.zip"

replace UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss


from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


# get the features from the file features.txt
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]
print('No of Features: {}'.format(len(features)))

No of Features: 561


In [2]:
## get the data from txt files to pandas dataffame
# training data
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
X_train.columns = features
X_train['Activity'] = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])

# training labels
y_train_subject = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', names=['subject'])
X_train['subject'] = y_train_subject

<ipython-input-2-76fd5a8203ee>:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)


In [3]:
# test data
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
X_test.columns = features
X_test['Activity'] = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])

# test labels
y_test_subject = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', names=['Activity'])
X_test['subject'] = y_test_subject

<ipython-input-3-abe73ffddcd5>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)


In [4]:
# Combine both train and test data
all_X_data = pd.concat([X_train, X_test], axis=0)
all_X_data = all_X_data.reset_index(drop=True)

all_y_data = pd.concat([y_train_subject, y_test_subject], axis=0)
all_y_data = all_y_data.reset_index(drop=True)

In [5]:
# Split the test train data
subject_train, subject_test = train_test_split(all_X_data, test_size=0.2, random_state=42)


In [6]:
y_train = subject_train['subject']
X_train = subject_train.drop('subject', axis=1)

y_test = subject_test['subject']
X_test = subject_test.drop('subject', axis=1)

In [14]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)


test_accuracy = lr_model.score(X_test, y_test)
test_loss = log_loss(y_test, sgd_clf.predict_proba(X_test))
print ("Accuracy and Loss for test data using LogisticRegression is {} and {}".format(test_accuracy, test_loss))

# Evaluate the model
train_accuracy = lr_model.score(X_train, y_train)
train_loss = log_loss(y_train, sgd_clf.predict_proba(X_train))
print ("Accuracy and Loss for training data using LogisticRegression is {} and {}".format(train_accuracy, train_loss))

Accuracy and Loss for test data using LogisticRegression is 0.6810679611650485 and 12.912178337473698
Accuracy and Loss for training data using LogisticRegression is 0.8581138487680544 and 9.817336993659847


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an SGDClassifier instance
sgd_clf = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)

# Fit the model
sgd_clf.fit(X_train, y_train)

# Make predictions
y_pred = sgd_clf.predict(X_test)

# Evaluate the model
accuracy = sgd_clf.score(X_test, y_test)
loss = log_loss(y_test, sgd_clf.predict_proba(X_test))
print ("Accuracy and Loss using LogisticRegression is {} and {}".format(accuracy, loss))

Accuracy and Loss using LogisticRegression is 0.5597087378640777 and 12.912178337473698


In [11]:
train_accuracy = sgd_clf.score(X_train, y_train)
train_loss = log_loss(y_train, sgd_clf.predict_proba(X_train))
print ("Accuracy and Loss for training data using LogisticRegression is {} and {}".format(train_accuracy, train_loss))

Accuracy and Loss for training data using LogisticRegression is 0.6585750697900231 and 9.817336993659847
